In [9]:
from datasets import load_dataset
import numpy as np

train_ds = load_dataset("Bisher/ASVspoof_2019_LA", split="train")

print("Total samples:", len(train_ds))
print("Speakers:", len(train_ds.unique("speaker_id")))

N_REAL = 200
N_SPOOF = 200

balanced_indices = []

for spk in train_ds.unique("speaker_id"):
    spk_indices = np.where(np.array(train_ds["speaker_id"]) == spk)[0]

    spk_subset = train_ds.select(spk_indices)

    real_indices = np.where(np.array(spk_subset["key"]) == 1)[0]

    spoof_indices = np.where(np.array(spk_subset["key"]) == 0)[0]

    real_sample = np.random.choice(
        real_indices, size=min(N_REAL, len(real_indices)), replace=False
    )

    spoof_sample = np.random.choice(
        spoof_indices, size=min(N_SPOOF, len(spoof_indices)), replace=False
    )

    balanced_indices.extend(spk_indices[real_sample])
    balanced_indices.extend(spk_indices[spoof_sample])

balanced_train_ds = train_ds.select(balanced_indices)

print("Balanced size:", len(balanced_train_ds))
print(balanced_train_ds.features)

/home/SpeakerRec/BioVoice/.venv_asv/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Total samples: 25380
Speakers: 20
Balanced size: 6580
{'speaker_id': Value(dtype='string', id=None), 'audio_file_name': Value(dtype='string', id=None), 'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None), 'system_id': Value(dtype='string', id=None), 'key': ClassLabel(names=['bonafide', 'spoof'], id=None)}


In [10]:
from pathlib import Path

PROJECT_ROOT = Path("/home/SpeakerRec/BioVoice")

SAVE_PATH = (
    PROJECT_ROOT / "data" / "datasets" / "asv_spoof_2019" / "balanced_train_asvspoof"
)

balanced_train_ds.save_to_disk(SAVE_PATH)

print("Saved to:", SAVE_PATH)

Saving the dataset (1/1 shards): 100%|██████████| 6580/6580 [00:01<00:00, 5129.96 examples/s]

Saved to: /home/SpeakerRec/BioVoice/data/datasets/asv_spoof_2019/balanced_train_asvspoof
